# Install Spark

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
The system cannot find the path specified.
'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open '$SPARK_VERSION-bin-hadoop2.7.tgz'


Exception: Unable to find py4j, your SPARK_HOME may not be configured correctly

In [ ]:
# Download Postgres driver
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

In [ ]:
# Create Spark session and add driver to Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

# Extract Data

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
# Read in presidential voting data by US county
url ="https://migration-election-impact.s3.us-east-2.amazonaws.com/Resources/countypres_2000-2016.csv"
spark.sparkContext.addFile(url)
county_pres_df = spark.read.csv(SparkFiles.get("countypres_2000-2016.csv"), sep=",", header=True, inferSchema=True)

In [ ]:
# Read in 2010 county migration data
url ="https://migration-election-impact.s3.us-east-2.amazonaws.com/Resources/county-to-county-2006-2010-current-residence-sort.xls"
spark.sparkContext.addFile(url)
migration_2010_df = spark.read.csv(SparkFiles.get("county-to-county-2006-2010-current-residence-sort.xls"), sep=",", header=True, inferSchema=True)

In [ ]:
#### Add additional data sets #####

# Transform Data

In [ ]:
### Merge & Transform data into clean DataFrame #####

# Load data into RDS

In [ ]:
from config import password
# Configure settings for RDS
mode = "overwrite"
jdbc_url="jdbc:postgresql://presidential-migration-db.cexjpewgnljl.us-east-2.rds.amazonaws.com:5432/presidential-migration-db"
config = {"user":"postgres",
          "password": f'{password}',
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to <table_name> in RDS
<dataframe_name>.write.jdbc(url=jdbc_url, table='<table_name>', mode=mode, properties=config)